In [2]:
from google.colab import files
uploaded = files.upload()

Saving archive (2).zip to archive (2).zip


In [3]:
import zipfile
import os

# Unzip the dataset
zip_path = 'archive (2).zip'
unzip_dir = 'emotion_dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# List the contents of the extracted directory
for root, dirs, files in os.walk(unzip_dir):
    print(root, len(dirs), "directories", len(files), "files")


emotion_dataset 0 directories 44 files


In [4]:
import librosa
import numpy as np

def extract_features(file_path, max_pad_len=174):
    try:
        audio, sr = librosa.load(file_path, sr=None, res_type='kaiser_fast')
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        pad_width = max_pad_len - mfcc.shape[1]
        if pad_width > 0:
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfcc = mfcc[:, :max_pad_len]
        return mfcc
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [5]:
import os
from sklearn.preprocessing import LabelEncoder

features = []
labels = []

# Traverse through the dataset directory to collect features and labels
for root, _, file_names in os.walk(unzip_dir):
    for file_name in file_names:
        if file_name.endswith('.wav'):  # Ensure it's a .wav file
            file_path = os.path.join(root, file_name)
            emotion = file_name.split('-')[2]  # Extracting emotion label from filename
            feature = extract_features(file_path)
            if feature is not None:
                features.append(feature)
                labels.append(emotion)

# Convert to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels to numerical values
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Reshape features to add a channel dimension for CNN input
features = np.expand_dims(features, axis=-1)

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(features.shape[1], features.shape[2], 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(np.unique(labels_encoded)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 38, 172, 32)         │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 38, 172, 32)         │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 19, 86, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 19, 86, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 17, 84, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 17, 84, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 42, 64)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 8, 42, 64)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 6, 40, 128)          │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 6, 40, 128)          │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 3, 20, 128)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 3, 20, 128)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 7680)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         983,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,078,022 (4.11 MB)

 Trainable params: 1,077,318 (4.11 MB)

 Non-trainable params: 704 (2.75 KB)

In [7]:
# Fit the model
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    verbose=1)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.1179 - loss: 3.0681 - val_accuracy: 0.0000e+00 - val_loss: 7.5665
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 446ms/step - accuracy: 0.2253 - loss: 2.4912 - val_accuracy: 0.2222 - val_loss: 7.5985
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 340ms/step - accuracy: 0.2652 - loss: 2.5777 - val_accuracy: 0.2222 - val_loss: 8.0040
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.5598 - loss: 1.1924 - val_accuracy: 0.2222 - val_loss: 8.1128
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.2652 - loss: 1.9401 - val_accuracy: 0.2222 - val_loss: 7.9801
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.5286 - loss: 1.4374 - val_accuracy: 0.1111 - val_loss: 7.8514
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.4714 - loss: 1.3640 - val_accuracy: 0.1111 - val_loss: 7.6811
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.4905 - loss: 1.3996 - val_accuracy: 0.2222 - val_loss

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc * 100:.2f}%")


1/1 - 0s - 75ms/step - accuracy: 0.4444 - loss: 2.2128
Test accuracy: 44.44%


In [ ]:
#app tutor